In [41]:
import numpy as np
import pandas as pd


In [42]:
df=pd.read_csv('csv_data/final_dataset.csv')
df.head()


,Tweet Id,Tweets,User Id,Screen Name,Class
0,319399851215433000,Oh yeah Colin! Smash those girls! :D #MKR,949380854,CreatrixKali,sexism
1,320817818222358000,It's insane they keep bringing people back. Wh...,297877558,quincepoacher,none
2,324114200450437000,@berkeley_eagle #MKR this shit show has more ...,272704749,MarkTramby,none
3,326286656854454000,*sigh* oh Colin 😍 #MKR,71416635,BinnyD,none
4,381988216292655000,"RT @brian_day15: I swear, I'm not sexist, but ...",930620467,YesYoureSexist,sexism


In [43]:
df.rename(columns={'Class':'Label'},inplace=True)

In [44]:
df.head()

,Tweet Id,Tweets,User Id,Screen Name,Label
0,319399851215433000,Oh yeah Colin! Smash those girls! :D #MKR,949380854,CreatrixKali,sexism
1,320817818222358000,It's insane they keep bringing people back. Wh...,297877558,quincepoacher,none
2,324114200450437000,@berkeley_eagle #MKR this shit show has more ...,272704749,MarkTramby,none
3,326286656854454000,*sigh* oh Colin 😍 #MKR,71416635,BinnyD,none
4,381988216292655000,"RT @brian_day15: I swear, I'm not sexist, but ...",930620467,YesYoureSexist,sexism


In [45]:
df['Label']=df['Label'].replace({'sexism':1,'none':0})

In [46]:
df.head()

,Tweet Id,Tweets,User Id,Screen Name,Label
0,319399851215433000,Oh yeah Colin! Smash those girls! :D #MKR,949380854,CreatrixKali,1
1,320817818222358000,It's insane they keep bringing people back. Wh...,297877558,quincepoacher,0
2,324114200450437000,@berkeley_eagle #MKR this shit show has more ...,272704749,MarkTramby,0
3,326286656854454000,*sigh* oh Colin 😍 #MKR,71416635,BinnyD,0
4,381988216292655000,"RT @brian_day15: I swear, I'm not sexist, but ...",930620467,YesYoureSexist,1


In [47]:
df=df.drop(['User Id','Screen Name','Tweet Id'],axis=1)

In [48]:
df.head()


,Tweets,Label
0,Oh yeah Colin! Smash those girls! :D #MKR,1
1,It's insane they keep bringing people back. Wh...,0
2,@berkeley_eagle #MKR this shit show has more ...,0
3,*sigh* oh Colin 😍 #MKR,0
4,"RT @brian_day15: I swear, I'm not sexist, but ...",1


In [49]:
df.tail()

,Tweets,Label
11320,"Oh fuck me hard with a rusty chainsaw, another...",0
11321,OMG SHUT UP DRASKO AND BIANCA #MKR #FINALFIVE,0
11322,STFU drasko #MKR,0
11323,Ash found her inner bogun #whistle #fingersint...,1
11324,Sorry but #JessieJ on #thevoiceau is being a r...,1


In [50]:
import re
import nltk
import multiprocessing
from transformers import BertTokenizerFast
from concurrent.futures import ProcessPoolExecutor

In [51]:

from nltk.corpus import stopwords
num_processes = multiprocessing.cpu_count()
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


In [52]:

# Remove stopwords
stopwords_set = set(nltk.corpus.stopwords.words('english'))

def remove_pattern(input_text, pattern):
    return re.sub(pattern, '', input_text)

def preprocess_text(text):
    # Remove specific characters
    processed_text = re.sub(r'[@#]', '', text)

    # Tokenize the text
    tokens = tokenizer.tokenize(processed_text)

    # Remove stopwords
    tokens = [token for token in tokens if token.lower() not in stopwords_set]

    # Convert tokens back to text
    processed_text = ' '.join(tokens)

    # Remove specific characters
    processed_text = remove_pattern(processed_text, r'!|rt|\*|:')

    return processed_text

def preprocess_tweets_parallel(tweets):
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        preprocessed_tweets = list(executor.map(preprocess_text, tweets))
    return preprocessed_tweets

# Split the tweets into chunks for parallel processing
chunk_size = len(df) // num_processes
tweet_chunks = [df['Tweets'][i:i+chunk_size] for i in range(0, len(df), chunk_size)]

# Preprocess tweets in parallel
preprocessed_tweets = []
for chunk in tweet_chunks:
    preprocessed_chunk = preprocess_tweets_parallel(chunk)
    preprocessed_tweets.extend(preprocessed_chunk)

# Assign preprocessed tweets back to the DataFrame
df['Tweets'] = preprocessed_tweets


In [53]:
df['Tweets'] = df['Tweets'].str.replace('#', '')

In [54]:
df['Tweets']=df['Tweets'].str.lower().str.split()


In [55]:
df['Tweets'] = df['Tweets'].apply(lambda x: ' '.join([word for word in x if word not in stopwords_set]))
# Remove specific characters
df['Tweets'] = df['Tweets'].apply(lambda x: remove_pattern(x, r'@\w+|#\w+|!|rt|\*|:|'))


In [56]:
from nltk.stem import	WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [57]:
df

,Tweets,Label
0,oh yeah colin smash girls mk r,1
1,' insane keep bringing people back . show end ...,0
2,berkeley _ eagle mk r shit show comeback johnn...,0
3,sigh oh colin [unk] mk r,0
4,"brian _ day 15 swear , ' sex ist , honestly ca...",1
...,...,...
11320,"oh fuck hard rusty chains aw , another round i...",0
11321,om g shut dr ask bianca mk r final fi,0
11322,st fu dr ask mk r,0
11323,ash found inner bog un whistle fingers int hem...,1


In [58]:

for i in df['Tweets']:
    for word in i:
        word=lemmatizer.lemmatize(word)

In [59]:
df.shape

(11325, 2)

In [60]:
X = df['Tweets'].astype(str)
Y = df['Label'].replace('Positive', 'Not Positive')

In [62]:
from collections import Counter

all_tweets = ' '.join(df['Tweets'])
words = all_tweets.split()
words_counts = Counter(words)

for word, count in words_counts.items():
    print(f"{word}: {count}")

oh: 321
yeah: 119
colin: 166
smash: 7
girls: 403
mk: 4561
r: 4988
': 6178
insane: 9
keep: 92
bringing: 15
people: 435
back: 190
.: 15277
show: 146
end: 74
berkeley: 3
_: 1627
eagle: 1
shit: 210
comeback: 9
johnny: 8
far: 78
nham: 3
,: 4301
ok: 127
vo: 54
ice: 52
au: 44
sigh: 14
[unk]: 272
brian: 12
day: 112
15: 364
swear: 30
sex: 1105
ist: 1168
honestly: 44
cannot: 67
stand: 76
woman: 237
college: 9
football: 94
announcer: 27
espn: 16
2: 437
j: 483
bro: 41
ml: 13
ey: 32
29: 26
science: 12
good: 300
says: 54
men: 397
mono: 8
gam: 5
ous: 24
amir: 7
ite: 19
?: 2012
dar: 15
km: 27
ch: 156
ris: 34
call: 257
think: 412
women: 781
seriously: 102
lacking: 2
knowledge: 6
l: 560
comes: 34
feminism: 89
random: 21
hannah: 3
cam: 17
ille: 4
20: 360
females: 50
really: 348
need: 234
stop: 200
acting: 11
like: 724
big: 93
fans: 12
yan: 4
di: 62
isa: 20
mine: 23
place: 48
work: 167
rock: 27
drill: 2
er: 247
practical: 2
lex: 2
na: 89
ija: 1
last: 104
wee: 274
want: 275
equal: 50
praise: 2
success: 6
g

'oh yeah colin smash girls mk r \' insane keep bringing people back . show end mk r berkeley _ eagle mk r shit show comeback johnny far nham , ok back vo ice au sigh oh colin [unk] mk r brian _ day 15 swear , \' sex ist , honestly cannot stand woman college football announcer espn 2 j bro ml ey 29 science good says men \' mono gam ous shit , amir ite ? ? dar km ch ris call sex ist think women seriously lacking knowledge l comes feminism random hannah cam ille 20 call sex ist , females really need stop acting like big football fans . stop . yan di isa _ mine place woman work . . . call sex ist woman rock drill er practical . lex _ na ija \' call sex ist last wee . women want equal praise success , gotta take equal blame fail ur … two th ug 4 u \' sorry . \' see \' hot girls eat like guys . call sex ist , like girls \' girls . every day think maybe day find sex ist wee ts , never day . keep good fight , women . two th ug 4 u yes res ex ist honestly stereotypes light - heaed usually harml

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [68]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
Y_test = Y_test.astype(str)

In [69]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [70]:
from sklearn.preprocessing import LabelEncoder

# Create label encoder object
label_encoder = LabelEncoder()

# Check if labels are already numeric
if isinstance(Y_train.values[0], str):
    # Encode string labels
    Y_train_encoded = label_encoder.fit_transform(Y_train.values)
else:
    # Convert numeric labels to strings and then encode
    Y_train_str = [str(label) for label in Y_train.values]
    Y_train_encoded = label_encoder.fit_transform(Y_train_str)

# Fit the logistic regression model
model = LogisticRegression()
model.fit(X_train_vec, Y_train_encoded)

/home/yash/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [71]:
Y_pred = model.predict(X_test_vec)

In [72]:
Y_test_encoded = label_encoder.transform(Y_test) 
print(classification_report(Y_test_encoded, Y_pred))

              precision    recall  f1-score   support

           0       0.89      0.94      0.91      1645
           1       0.82      0.68      0.75       619
           2       0.00      0.00      0.00         1

    accuracy                           0.87      2265
   macro avg       0.57      0.54      0.55      2265
weighted avg       0.87      0.87      0.87      2265

